In [ ]:
!pip install essentia

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.6/13.6 MB 13.9 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import essentia.standard as es
import os

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Label each frame for evaluation

In [ ]:
sr = 44100
frame_length =  16384
step_size = int(frame_length/2)
frame_time = step_size/sr

directory = "/content/drive/MyDrive/ColabNotebooks/muspeak/"

for filename in os.listdir(directory):
    if filename.startswith('track') and filename.endswith('.csv'):
      df_segments = pd.read_csv(directory + filename)
      numOfSegments = len(df_segments.index) - 1
      audio_lenght = df_segments['t1'][numOfSegments] + df_segments['t2'][numOfSegments]
      df_classes = pd.DataFrame({"t1":[],"t2":[], "class":[]})
      time = 0
      while time < audio_lenght:
        df_class = pd.DataFrame({"t1":[time],"t2":[time + frame_time],"class":['silence']})
        time = time + frame_time
        df_classes = df_classes.append(df_class, ignore_index=True)
      for index, row in df_segments.iterrows():
        numOfFrames = round(row['t2']/frame_time)
        start_frame = round(row['t1']/frame_time)
        for i in range(numOfFrames):
          if df_classes.loc[(start_frame + i), 'class'] == 'silence':
            df_classes.loc[(start_frame + i), 'class'] = row['class']
          else:
            df_classes.loc[start_frame + i, 'class'] = 's&m'
      pd.DataFrame(df_classes).to_csv(directory + 'labeled_' + filename)

In [ ]:
sr = 44100
frame_length =  16384/2

seg = pd.read_csv('/content/drive/MyDrive/ColabNotebooks/scottish/scottish.csv')
time = 0
segment = 0
length = seg['t2'].iloc[-1]
df_classes = pd.DataFrame({"t1":[],"t2":[], "class":[]})
while time < length:
  if(time + (frame_length/sr) < length):
    df_class = pd.DataFrame({"t1":[time],"t2":[time + (frame_length/sr)],"class":[seg['class'][segment]]})
  else:
    df_class = pd.DataFrame({"t1":[time],"t2":[length],"class":[seg['class'][segment]]})
  time = time + frame_length/sr
  if(time >= seg['t2'][segment]):
    segment = segment + 1
  df_classes = df_classes.append(df_class, ignore_index=True)

pd.DataFrame(df_classes).to_csv('/content/drive/MyDrive/ColabNotebooks/scottish/scottish_labeled.csv')


Read Dataset, divide each track into frames and obtain the different descriptors from each frame. Put them in a dataframe with their corresponding class.

In [ ]:
step_size = int(frame_length/2)

audio_loader = es.MonoLoader(filename='/content/drive/MyDrive/ColabNotebooks/muspeak/scottish.wav')
audio_test = audio_loader()
mfcc = es.MFCC()
spectrum = es.Spectrum()
w = es.Windowing(type='hann')
dfm = pd.DataFrame()
for frame in es.FrameGenerator(audio_test, frameSize = frame_length, hopSize = step_size, startFromZero=True):
  mfcc_bands, mfcc_coeffs = mfcc(spectrum(w(frame)))
  df2 = pd.DataFrame({"mfcc1":[mfcc_coeffs[0]],"mfcc2":[mfcc_coeffs[1]],"mfcc3":[mfcc_coeffs[2]],
                      "mfcc4":[mfcc_coeffs[3]],"mfcc5":[mfcc_coeffs[4]],"mfcc6":[mfcc_coeffs[5]],
                      "mfcc7":[mfcc_coeffs[6]],"mfcc8":[mfcc_coeffs[7]],"mfcc9":[mfcc_coeffs[8]],
                      "mfcc10":[mfcc_coeffs[9]],"mfcc11":[mfcc_coeffs[10]],"mfcc12":[mfcc_coeffs[11]],
                      "mfcc13":[mfcc_coeffs[12]]})
  dfm = dfm.append(df2, ignore_index=True)

In [ ]:
import joblib
model = joblib.load('/content/drive/MyDrive/ColabNotebooks/first_model_20000.pkl')
model.predict(dfm)

/usr/local/lib/python3.8/dist-packages/sklearn/base.py:299: UserWarning: Trying to unpickle estimator SVC from version 1.0.2 when using version 1.2.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


array(['s', 's', 's', ..., 's', 's', 's'], dtype=object)

In [ ]:
y_test = model.predict(dfm)
pd.DataFrame(y_test).to_csv('/content/drive/MyDrive/ColabNotebooks/y_test_20000.csv')

In [ ]:
import math
dfmi = pd.read_csv('/content/drive/MyDrive/ColabNotebooks/music_mfcc_ground_truth.csv')
dfsi = pd.read_csv('/content/drive/MyDrive/ColabNotebooks/speech_mfcc_ground_truth.csv')
np.random.seed(10)
n_rm_music = dfmi.shape[0]-5000
n_rm_speech = dfsi.shape[0]-5000
i_rm_music = np.random.choice(dfmi.index, n_rm_music, replace=False)
i_rm_speech = np.random.choice(dfsi.index, n_rm_speech, replace=False)
dfmi = dfmi.drop(i_rm_music)
dfsi = dfsi.drop(i_rm_speech)
dfall = dfmi.head().append(dfsi.head())

In [ ]:
dfall = dfmi.append(dfsi)

In [ ]:
X = dfall.drop('class',axis=1)
y = dfall['class']

In [ ]:
from sklearn import svm
clf = svm.SVC(kernel='linear')
clf.fit(X,y)
y_pred = clf.predict(X)
y_pred

In [ ]:
import joblib
joblib.dump(clf,'/content/drive/MyDrive/ColabNotebooks/first_model_5000.pkl',compress=9)

['/content/drive/MyDrive/ColabNotebooks/first_model_5000.pkl']

https://stackabuse.com/implementing-svm-and-kernel-svm-with-pythons-scikit-learn/
https://github.com/ovh/ai-training-examples/blob/main/notebooks/audio/audio-classification/notebook-marine-sound-classification.ipynb
https://essentia.upf.edu/streaming_extractor_music.html
https://www.datacamp.com/tutorial/svm-classification-scikit-learn-python
https://stackabuse.com/implementing-svm-and-kernel-svm-with-pythons-scikit-learn/
https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.drop.html
